In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#loading mnist
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#define batch size
batch_size = 128
n_batch = mnist.train.num_examples // batch_size

#define summaries
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#standard deviation
        tf.summary.scalar('max',tf.reduce_max(var))#maximum
        tf.summary.scalar('min',tf.reduce_min(var))#minimum
        tf.summary.histogram('histogram',var)#histogram

#define name scope
with tf.name_scope('input'):
    #define placeholder
    x = tf.placeholder(tf.float32,[None,784], name='x-input')
    y = tf.placeholder(tf.float32,[None,10], name='y-input')
    
with tf.name_scope('layer'):
    with tf.name_scope('keep_prob'):
        keep_prob=tf.placeholder(tf.float32, name='keep_prob')
        variable_summaries(keep_prob)
    with tf.name_scope('lr'):
        lr = tf.Variable(1,dtype=tf.float32, name='lr')
    with tf.name_scope('layer1'):
        with tf.name_scope('weights1'):
            W1 = tf.Variable(tf.truncated_normal([784,300],stddev=0.1), name='weights1')
            variable_summaries(W1)
        with tf.name_scope('biases1'):
            b1 = tf.Variable(tf.zeros([1,300]),name='b1')
            variable_summaries(b1)
        with tf.name_scope('layer1Relu'):
            L1 = tf.nn.relu(tf.matmul(x,W1)+b1)
        with tf.name_scope('layer1Drop'):
            L1_drop = tf.nn.dropout(L1,keep_prob)
    with tf.name_scope('layer2'):
        with tf.name_scope('weights2'):
            W2 = tf.Variable(tf.truncated_normal([300,100],stddev=0.1), name='weights2')
            variable_summaries(W2)
        with tf.name_scope('biases2'):
            b2 = tf.Variable(tf.zeros([1,100]),name='b2')
            variable_summaries(b2)
        with tf.name_scope('layer2Relu'):
            L2 = tf.nn.relu(tf.matmul(L1_drop,W2)+b2)
        with tf.name_scope('layer2Drop'):
            L2_drop = tf.nn.dropout(L2,keep_prob)
    with tf.name_scope('layer3'):
        with tf.name_scope('weights3'):
            W3 = tf.Variable(tf.truncated_normal([100,10],stddev=0.1), name='weights3')
            variable_summaries(W3)
        with tf.name_scope('biases3'):
            b3 = tf.Variable(tf.zeros([1,10]),name='b3')
            variable_summaries(b3)
    with tf.name_scope('prediction1'):
        prediction = tf.matmul(L2_drop,W3) + b3
    with tf.name_scope('prediction2'):
        prediction_2 = tf.nn.softmax(prediction)

#prediction = tf.nn.softmax(tf.matmul(L2_drop,W3) + b3)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.AdadeltaOptimizer(lr).minimize(loss)

#init
init = tf.global_variables_initializer()

#result maintain(return most possible number)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction_2,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)                             

#combind all summary
merged = tf.summary.merge_all()
                             
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(10):
        sess.run(tf.assign(lr, 1*(0.98**epoch)))
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.8})
        
        writer.add_summary(summary,epoch)                     
        learning_rate = sess.run(lr)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0} )
        print("Iter " + str(epoch) + " Testing Accuracy: " + str(acc)+ ",  lr: "+ str(learning_rate))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter 0 Testing Accuracy: 0.9337,  lr: 1.0
Iter 1 Testing Accuracy: 0.9514,  lr: 0.98
Iter 2 Testing Accuracy: 0.9594,  lr: 0.9604
Iter 3 Testing Accuracy: 0.9646,  lr: 0.941192
Iter 4 Testing Accuracy: 0.9684,  lr: 0.92236817
Iter 5 Testing Accuracy: 0.9717,  lr: 0.90392077
Iter 6 Testing Accuracy: 0.9728,  lr: 0.8858424
Iter 7 Testing Accuracy: 0.9751,  lr: 0.86812556
Iter 8 Testing Accuracy: 0.9748,  lr: 0.850763
Iter 9 Testing Accuracy: 0.9755,  lr: 0.83374774
